In [34]:
from langchain.prompts import PromptTemplate

template = """
You are a medical assistant chatbot. You have access to previous conversations between doctors and patients. Here is the context of those conversations:

Context: {context}

A new patient is asking the following question:
Context: {question}

Based on the provided conversations, respond to the patient's query by using relevant information. Ensure that your response is clear, informative, and empathetic.
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are a medical assistant chatbot. You have access to previous conversations between doctors and patients. Here is the context of those conversations:

Context: Here is some context

A new patient is asking the following question:
Context: Here is a question

Based on the provided conversations, respond to the patient's query by using relevant information. Ensure that your response is clear, informative, and empathetic.



# define a API retriever

In [35]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.insert(0, parent_dir)

# Now you can import modules from the parent directory
from utils.APIs import API_retriever

# Create an instance of API_retriever
retriever = API_retriever("http://localhost:8002/query")

# Construct the path to the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.insert(0, parent_dir)
from utils.APIs import API_retriever
retriever=API_retriever("http://localhost:8002/query")

In [36]:
# retriever.invoke("I am sick?")

In [37]:
#define API LLM

# 2. Build RAG Chain (Optional)

In [38]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = prompt | model | StrOutputParser()

In [39]:
def answer_fn(question, history=None):

    context=retriever.invoke(question)['contexts']
    return chain.invoke({"question": question,"context":context})

In [40]:
# print(answer_fn("I am sick."))

# 3. New evaluate the RAG

In [41]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = DirectoryLoader('../../Consultations20240916', glob="**/*.txt")
documents = loader.load_and_split(text_splitter)
print(documents)

[Document(metadata={'source': '../../Consultations20240916/Lawrence_320361.txt'}, page_content="Doctor: Lawrence On\n\nPatient: Lauren Demers, F, 34, Allergies:\n\nPast medications:\n\nCondition description: full body rash;\n\nPhone transcript: Dr: Hi Lauren, my name is Lauren, I'm one of the doctors. How are you?\n\nPt: Yeah, where are you?\n\nDr: Okay, do you want to send me a photo of your rash?\n\nPt: Yep, how do I do that?\n\nDr: If you click on the link, it will take you to our website.\n\nPt: Sorry, did I lose you?\n\nDr: Yeah, if you click on the link, it will take you to our webpage where you can send it.\n\nPt: Oh, I see it, okay.\n\nDr: Have you been sick recently?\n\nPt: Nope.\n\nDr: New medications?\n\nPt: Nope.\n\nDr: Okay, yeah it just looks like a urticaria is what we call it. Have you taken any antihistamines?\n\nPt: Yeah.\n\nDr: Okay. Is it quite itchy?\n\nPt: It's so itchy.\n\nDr: Okay, we'll probably start you on some steroids.\n\nPt: Okay.\n\nDr: So it's just two t

In [42]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

In [43]:
from giskard.rag import KnowledgeBase
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)
knowledge_base = KnowledgeBase(df)

In [44]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/60 [00:00<?, ?it/s]

CorrectnessMetric evaluation: 100%|██████████| 60/60 [02:28<00:00,  2.48s/it]


In [45]:
display(report)

Loading BokehJS ...

2024-09-16 21:47:30,479 pid:2551721 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-09-16 21:47:38,915 pid:2551721 MainThread giskard.rag  INFO     Found 4 topics in the knowledge base.


In [48]:
report.to_html("OpenAI_RAG_1000_report.html")

# 4. Evaluate Without RAG

In [49]:
from langchain.prompts import PromptTemplate

non_rag_template = """
You are a medical assistant chatbot. You have access to previous conversations between doctors and patients.

The patient is asking the following question:
Context: {question}

Respond to the patient's query, ensuring that your response is clear, informative, and empathetic.
"""

non_rag_prompt = PromptTemplate.from_template(non_rag_template)
print(non_rag_prompt.format(context="Here is some context", question="Here is a question"))


You are a medical assistant chatbot. You have access to previous conversations between doctors and patients.

The patient is asking the following question:
Context: Here is a question

Respond to the patient's query, ensuring that your response is clear, informative, and empathetic.



In [51]:
non_rag_chain = non_rag_prompt | model | StrOutputParser()

In [52]:
def non_rag_answer_fn(question, history=None):
    return non_rag_chain.invoke({"question": question})

In [53]:
print(non_rag_answer_fn("I am sick."))

I'm sorry to hear that you're feeling sick. It's important to take care of yourself when you're not feeling well. Make sure to rest, stay hydrated, and consider contacting your doctor for further evaluation and treatment. If you have any specific symptoms or concerns, please let me know so I can provide you with the appropriate guidance. Remember, your health is important, so don't hesitate to seek medical help if needed. Take care.


In [54]:

non_rag_report = evaluate(non_rag_answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [02:51<00:00,  2.86s/it]


In [55]:
display(non_rag_report)

Loading BokehJS ...

In [56]:
non_rag_report.to_html("OpenAI_NO_RAG_report.html")